# Intro to Gragh Convolutional Surrogate Models
Eamon Whalen

In [1]:
import sys
import numpy as np
import pandas as pd
import altair as alt

sys.path.append('./models')
from feastnetSurrogateModel import FeaStNet

sys.path.append('./readers')
from loadGhGraphs import loadGhGraphs

sys.path.append('./visualization')
from altTrussViz import plotTruss, interactiveErrorPlot

sys.path.append('./util')
from gcnSurrogateUtil import *

## 1. Load simulation data

In [2]:
doeFile = "/home/ewhalen/projects/data/trusses/2D_Truss_v1.3/design_7_N_1000.csv"
allGraphsUnfiltered = loadGhGraphs(doeFile, NUM_DV=5)

maxes = [max(np.abs(graph.y.numpy().flatten())) for graph in allGraphsUnfiltered]
source = pd.DataFrame(maxes, columns=['maxes'])
source.describe()

,maxes
count,1000.000000
mean,0.199516
std,2.524026
min,0.006570
25%,0.015425
50%,0.024334
75%,0.046981
max,67.325867


In [3]:
plotTruss(allGraphsUnfiltered[0], showDeformed=True, defScale=100)

alt.LayerChart(...)

## 2. Filter and partition

In [4]:
allGraphs = filterbyDisp(allGraphsUnfiltered, 0.9)
trainData, valData, testData = partitionGraphList(allGraphs)

maxes = [max(np.abs(graph.y.numpy().flatten())) for graph in allGraphs]
source = pd.DataFrame(maxes, columns=['maxes'])
source.describe()

,maxes
count,900.000000
mean,0.028952
std,0.019071
min,0.006570
25%,0.014951
50%,0.021719
75%,0.038367
max,0.097861


## 3. Train a GCN

In [5]:
gcn = FeaStNet()
history = gcn.trainModel(trainData, valData, 
                     epochs=2, 
                     batch_size=256, 
                     flatten=True, 
                     logTrans=False, 
                     ssTrans=True, 
                     saveDir='./results/debugging/')

plotHistory(history)

epoch: 0   trainLoss: 9.8918e-01   valLoss:9.4729e-01  time: 5.22e+00
epoch: 1   trainLoss: 9.0822e-01   valLoss:9.4022e-01  time: 1.05e+00
loading checkpoint 1


alt.Chart(...)

## 4. Test the GCN

In [6]:
trainRes = gcn.testModel(trainData)
testRes = gcn.testModel(testData)
pd.DataFrame([trainRes, testRes], index=['train', 'test'])

,mse,mae,mre,peakR2,maxAggR2,meanAggR2,minAggR2
train,0.000210,0.009714,0.375235,-1.328127,-0.000199,-7.043110,-102.89973
test,0.000167,0.008687,0.351390,-1.367504,-0.000007,-5.542199,-79.72868


## 5. Visualize some predictions

In [7]:
i = 9
pred = gcn.predict([testData[i]])[0]
plotTruss(testData[i], showDeformed=True, defScale=100, prediction=pred)

alt.LayerChart(...)

In [9]:
alt.data_transformers.enable('json')
allPreds = gcn.predict(testData)
interactiveErrorPlot(testData, allPreds)

,x,y,design,bar,type
0,0.000000,0.000000,0.0,0.0,undeformed
1,2.667094,-11.132656,0.0,0.0,undeformed
2,2.612203,0.726291,0.0,1.0,deformed
3,3.374830,-10.669490,0.0,1.0,deformed
4,-0.324877,-0.553457,0.0,2.0,prediction
...,...,...,...,...,...
29155,50.000000,0.000000,179.0,14577.0,undeformed
29156,47.181442,-10.084947,179.0,14578.0,deformed
29157,49.752567,-0.596296,179.0,14578.0,deformed
29158,47.046074,-10.197928,179.0,14579.0,prediction


,mse,mae,mre,maxAE,mae/peak,maxAE/peak,relEAtPeak
0,0.000937,0.025846,0.683344,0.068818,0.399108,1.062657,0.893139
1,0.000019,0.003542,0.186996,0.009922,0.210385,0.589311,0.589054
2,0.000015,0.002976,0.158188,0.007725,0.203296,0.527700,0.527355
3,0.000033,0.004705,0.250285,0.011263,0.258853,0.619661,0.619333
4,0.000050,0.005789,0.296386,0.017796,0.234295,0.720247,0.719916
...,...,...,...,...,...,...,...
175,0.000260,0.013640,0.545286,0.035804,0.429628,1.127761,0.782086
176,0.000019,0.003220,0.181241,0.010751,0.182276,0.608529,0.608329
177,0.000046,0.005500,0.265663,0.016005,0.274104,0.797669,0.655173
178,0.000025,0.004115,0.206633,0.011309,0.225761,0.620469,0.620408


alt.HConcatChart(...)